In [1]:
import numpy as np

In [2]:
#extracting data from csv files into numpy arrays
from numpy import genfromtxt
x_train = genfromtxt('train_data.csv', delimiter=',')
y_train = genfromtxt('train_labels.csv', delimiter=',')
x_test = genfromtxt('test_data.csv', delimiter=',')
y_test = genfromtxt('test_labels.csv', delimiter=',')

In [3]:
#shape
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((153, 40), (50, 40), (153,), (50,))

In [4]:
#converting to one hot
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)
y_train.shape,y_test.shape

((153, 10), (50, 10))

In [6]:
#reshaping to shape required by CNN
x_train=np.reshape(x_train,(x_train.shape[0], 40,1,1))
x_test=np.reshape(x_test,(x_test.shape[0], 40,1,1))

In [7]:
#shapes
x_train.shape,x_test.shape

((153, 40, 1, 1), (50, 40, 1, 1))

In [8]:
from tensorflow.keras import Sequential
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras import Model

2.2.0


In [9]:
#forming model
model=Sequential()

In [10]:
#adding layers and forming the model
model.add(Conv2D(64,kernel_size=5,strides=1,padding="Same",activation="relu",input_shape=(40,1,1)))
model.add(MaxPooling2D(padding="same"))

model.add(Conv2D(128,kernel_size=5,strides=1,padding="same",activation="relu"))
model.add(MaxPooling2D(padding="same"))
model.add(Dropout(0.3))

model.add(Flatten())

model.add(Dense(256,activation="relu"))
model.add(Dropout(0.3))

model.add(Dense(512,activation="relu"))
model.add(Dropout(0.3))

model.add(Dense(10,activation="softmax"))

In [11]:
#compiling
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [12]:
#training the model
model.fit(x_train,y_train,batch_size=50,epochs=30,validation_data=(x_test,y_test))

Epoch 1/30
4/4 [==============================] - 0s 48ms/step - loss: 2.0726 - accuracy: 0.2745 - val_loss: 3.3352 - val_accuracy: 0.1200
Epoch 2/30
4/4 [==============================] - 0s 13ms/step - loss: 1.6048 - accuracy: 0.4967 - val_loss: 3.6241 - val_accuracy: 0.1000
Epoch 3/30
4/4 [==============================] - 0s 13ms/step - loss: 1.3376 - accuracy: 0.5556 - val_loss: 4.1635 - val_accuracy: 0.1200
Epoch 4/30
4/4 [==============================] - 0s 13ms/step - loss: 1.0308 - accuracy: 0.7255 - val_loss: 4.1981 - val_accuracy: 0.1200
Epoch 5/30
4/4 [==============================] - 0s 13ms/step - loss: 0.7843 - accuracy: 0.7778 - val_loss: 4.3473 - val_accuracy: 0.1200
Epoch 6/30
4/4 [==============================] - 0s 12ms/step - loss: 0.7192 - accuracy: 0.7974 - val_loss: 4.4873 - val_accuracy: 0.1200
Epoch 7/30
4/4 [==============================] - 0s 13ms/step - loss: 0.5531 - accuracy: 0.8170 - val_loss: 4.0361 - val_accuracy: 0.1200
Epoch 8/30
4/4 [===========

In [13]:
#train and test loss and scores respectively
train_loss_score=model.evaluate(x_train,y_train)
test_loss_score=model.evaluate(x_test,y_test)
print(train_loss_score)
print(test_loss_score)

2/2 [==============================] - 0s 2ms/step - loss: 4.8164 - accuracy: 0.7400
[0.0033533996902406216, 1.0]
[4.816380023956299, 0.7400000095367432]


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 40, 1, 64)         1664      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 20, 1, 64)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 1, 128)        204928    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 1, 128)        0         
_________________________________________________________________
dropout (Dropout)            (None, 10, 1, 128)        0         
_________________________________________________________________
flatten (Flatten)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               3

In [15]:
model.save('urbanClassifier')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: urbanClassifier/assets


In [16]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TF Lite model.
with tf.io.gfile.GFile('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [17]:
x_test[0].shape

(40, 1, 1)

In [18]:
testData = np.expand_dims(x_test[5],axis=0)

In [19]:
prediction = model.predict_classes(testData)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [20]:
prediction

array([3])

In [21]:
y_test[5]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [22]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [23]:
input_details

[{'name': 'conv2d_input',
  'index': 0,
  'shape': array([ 1, 40,  1,  1], dtype=int32),
  'shape_signature': array([ 1, 40,  1,  1], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [26]:
testData = np.expand_dims(x_test[5],axis=0)
atData = np.float32(testData)

In [25]:
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], atData)
print(input_data.dtype)
print(testData.shape)
interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

float32
(1, 40, 1, 1)
[[7.1473175e-11 7.8754218e-11 7.2361290e-05 9.9991715e-01 4.1809503e-07
  2.0054536e-08 2.4804867e-06 4.1848831e-09 7.6419092e-06 3.6501409e-09]]
